# CT-LTI: Multiple Sample Performance Evaluation Table
This table is found in the appendix section A.4. and summarizes the performance comparison between NODEC and OC in relative terms of error and energy. Without extensive hyperparameter optimization we see that NODEC is competitive to OC for all graphs and intial-target state settings.

Furthermore, please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/ct_lti/gen_parameters.py```.

Please also make sure that a trainingproceedure has produced results in the corresponding paths used below.
Running ```nodec_experiments/ct_lti/multi_sample/train.ipynb``` with default paths is expected to generate at the requiered location.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

## Gather data from files
Below we gather the data from files generated by the ```train_and_eval.ipynb``` file. Please run this first if the data files are not present!

In [ ]:
data_folder = '../../../../data/results/ct_lti/multi_sample/'
graphs = ['lattice', 'ba', 'tree']
graph_name = {'lattice' : 'Square Lattice', 'ba' : 'Barabasi Albert', 'tree' : 'Random Tree'}
resulting_rows = []
for graph in graphs:
    graph_folder = data_folder + graph + '/'
    interactions = [50, 500, 5000]
    for interaction in interactions:
        mse_diffs = []
        energy_diffs = []
        for i in range(100):
            nnres = pd.read_csv(graph_folder+'nn_sample_'+str(i)+'_train_'+str(interaction)+'/epoch_metadata.csv')
            ocres = pd.read_csv(graph_folder+'oc_sample'+str(i)+'_ninter_'+str(interaction)+'/epoch_metadata.csv')
            nn_en = nnres['total_energy'].item()
            oc_en = ocres['total_energy'].item()
            nn_fl = nnres['final_loss'].item()
            oc_fl = ocres['final_loss'].item()
            mse_diffs.append((nn_fl-oc_fl)/oc_fl)
            energy_diffs.append((nn_en-oc_en)/oc_en)
        row = {'Graph' : graph_name[graph], 'Interaction Interval': 5.0/interaction, 
               'Median Energy' : round(np.quantile(energy_diffs, 0.5), 2),
               'IQR Energy'    : round(np.quantile(energy_diffs, 0.75)-np.quantile(energy_diffs,0.25), 2),
               'Median MSE'    : round(np.quantile(mse_diffs, 0.5), 2),
               'IQR MSE'       : round(np.quantile(mse_diffs, 0.75)-np.quantile(mse_diffs, 0.25), 2),
               'Numerical Instabilities' : round((np.array(mse_diffs) > 10).mean(), 2)
              }
        resulting_rows.append(row)

## Resulting Table

In [ ]:
df = pd.DataFrame(resulting_rows).groupby(['Graph', 'Interaction Interval']).first()

In [ ]:
styler = df.style.apply(lambda x: ["background: lightblue" if v <= 0.1 and i in [0,2] else "" for i,v in enumerate(x)], axis = 1)
styler